# Medical Appointment No Shows Capstone

### 4. Feature Engineering - PreProcessing - Training Data Development

In [43]:
#Import necessary libraries
import pandas as pd
import numpy as np

# Plotting modules
import matplotlib.pyplot as plt
import seaborn as sns

# File system manangement
import os,sys

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
%matplotlib notebook

In [44]:
# SKLEARN
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [45]:
path = 'data/step2_output.csv'
df = pd.read_csv(path, index_col=None, parse_dates=["ScheduledDay","AppointmentDay"]) 
df.head()

,PatientId,AppointmentId,Gender,ScheduledDay,AppointmentDay,Age,Neighborhood,Scholarship,Hypertension,Diabetes,...,Handicap,SMSSent,NoShow,WaitingDays,ScheduledTime,ScheduledHour,ScheduledDayWeekDay,AppointmentDayWeekName,ScheduledDayMonthName,AppointmentDayMonthName
0,29872499824296,5642903,F,2016-04-29 18:38:08,2016-04-29 18:38:08,62,JARDIM DA PENHA,0,1,0,...,0,0,No,0,18:38:08,18,Friday,Friday,April,April
1,558997776694438,5642503,M,2016-04-29 16:08:27,2016-04-29 16:08:27,56,JARDIM DA PENHA,0,0,0,...,0,0,No,0,16:08:27,16,Friday,Friday,April,April
2,4262962299951,5642549,F,2016-04-29 16:19:04,2016-04-29 16:19:04,62,MATA DA PRAIA,0,0,0,...,0,0,No,0,16:19:04,16,Friday,Friday,April,April
3,867951213174,5642828,F,2016-04-29 17:29:31,2016-04-29 17:29:31,8,PONTAL DE CAMBURI,0,0,0,...,0,0,No,0,17:29:31,17,Friday,Friday,April,April
4,8841186448183,5642494,F,2016-04-29 16:07:23,2016-04-29 16:07:23,56,JARDIM DA PENHA,0,1,1,...,0,0,No,0,16:07:23,16,Friday,Friday,April,April


In [46]:
df = df.drop(["PatientId","AppointmentId"],axis=1)

In [47]:
df['ScheduledDay_Y'] = df['ScheduledDay'].dt.year
df['ScheduledDay_D'] = df['ScheduledDay'].dt.day
df.drop(['ScheduledDay'], axis=1, inplace=True)

df['AppointmentDay_Y'] = df['AppointmentDay'].dt.year
df['AppointmentDay_D'] = df['AppointmentDay'].dt.day
df.drop(['AppointmentDay'], axis=1, inplace=True)

df.drop(['ScheduledTime'], axis=1, inplace=True)

In [48]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110517 entries, 0 to 110516
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Gender                   110517 non-null  object
 1   Age                      110517 non-null  int64 
 2   Neighborhood             110517 non-null  object
 3   Scholarship              110517 non-null  int64 
 4   Hypertension             110517 non-null  int64 
 5   Diabetes                 110517 non-null  int64 
 6   Alcoholism               110517 non-null  int64 
 7   Handicap                 110517 non-null  int64 
 8   SMSSent                  110517 non-null  int64 
 9   NoShow                   110517 non-null  object
 10  WaitingDays              110517 non-null  int64 
 11  ScheduledHour            110517 non-null  int64 
 12  ScheduledDayWeekDay      110517 non-null  object
 13  AppointmentDayWeekName   110517 non-null  object
 14  ScheduledDayMonthNam

In [49]:
df.select_dtypes(include=[object])

,Gender,Neighborhood,NoShow,ScheduledDayWeekDay,AppointmentDayWeekName,ScheduledDayMonthName,AppointmentDayMonthName
0,F,JARDIM DA PENHA,No,Friday,Friday,April,April
1,M,JARDIM DA PENHA,No,Friday,Friday,April,April
2,F,MATA DA PRAIA,No,Friday,Friday,April,April
3,F,PONTAL DE CAMBURI,No,Friday,Friday,April,April
4,F,JARDIM DA PENHA,No,Friday,Friday,April,April
...,...,...,...,...,...,...,...
110512,F,MARIA ORTIZ,No,Tuesday,Tuesday,May,June
110513,F,MARIA ORTIZ,No,Tuesday,Tuesday,May,June
110514,F,MARIA ORTIZ,No,Wednesday,Tuesday,April,June
110515,F,MARIA ORTIZ,No,Wednesday,Tuesday,April,June


---
# Sklearn's OneHotEncoder Approach

In [50]:
categorical_features = ['Gender','NoShow','Neighborhood',
                       'ScheduledDayWeekDay','AppointmentDayWeekName','ScheduledDayMonthName','AppointmentDayMonthName']



ohe = OneHotEncoder(sparse=True, drop='first', handle_unknown='error')


encoded = pd.DataFrame(ohe.fit_transform(df[categorical_features]).toarray(), \
                                columns=ohe.get_feature_names(categorical_features))

dropped_cols = df.drop(columns=categorical_features)


#df_ohe = data_hot_encoded_drop.join(data_hot_encoded)
df_ohe = pd.concat([dropped_cols,encoded],axis=1)

df_ohe

,Age,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSSent,WaitingDays,ScheduledHour,ScheduledDay_Y,...,AppointmentDayWeekName_Wednesday,ScheduledDayMonthName_December,ScheduledDayMonthName_February,ScheduledDayMonthName_January,ScheduledDayMonthName_June,ScheduledDayMonthName_March,ScheduledDayMonthName_May,ScheduledDayMonthName_November,AppointmentDayMonthName_June,AppointmentDayMonthName_May
0,62,0,1,0,0,0,0,0,18,2016,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,56,0,0,0,0,0,0,0,16,2016,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,0,0,0,0,0,0,0,16,2016,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,0,0,0,0,0,0,0,17,2016,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,56,0,1,1,0,0,0,0,16,2016,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110512,56,0,0,0,0,0,1,34,9,2016,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
110513,51,0,0,0,0,0,1,34,7,2016,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
110514,21,0,0,0,0,0,1,40,16,2016,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
110515,38,0,0,0,0,0,1,40,15,2016,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [54]:
# ohe.get_feature_names(input_features=categorical_features)

___

# LabelEncoder's Approach

### df_le1

In [8]:
df_le1 = df.loc[:, ['Gender', 'Neighborhood', 'NoShow']]

In [9]:
le = LabelEncoder()

df_le1['Gender'] = le.fit_transform(df_le1['Gender'])
df_le1['Neighborhood'] = le.fit_transform(df_le1['Neighborhood'])
df_le1['NoShow'] = le.fit_transform(df_le1['NoShow'])



df_le1.head()

,Gender,Neighborhood,NoShow
0,0,38,0
1,1,38,0
2,0,44,0
3,0,53,0
4,0,38,0


### df_le2

In [11]:
df_le2 = df.loc[:]

In [12]:
le = LabelEncoder()

df_le2['Gender'] = le.fit_transform(df_le2['Gender'])
df_le2['Neighborhood'] = le.fit_transform(df_le2['Neighborhood'])
df_le2['NoShow'] = le.fit_transform(df_le2['NoShow'])

# use df.apply() to apply le.fit_transform to all columns
#df = df.apply(le.fit_transform)

df_le2.head()

,Gender,Age,Neighborhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSSent,NoShow,WaitingDays,ScheduledDay_Y,ScheduledDay_M,ScheduledDay_D,AppointmentDay_Y,AppointmentDay_M,AppointmentDay_D
0,0,62,38,0,1,0,0,0,0,0,-1,2016,4,29,2016,4,29
1,1,56,38,0,0,0,0,0,0,0,-1,2016,4,29,2016,4,29
2,0,62,44,0,0,0,0,0,0,0,-1,2016,4,29,2016,4,29
3,0,8,53,0,0,0,0,0,0,0,-1,2016,4,29,2016,4,29
4,0,56,38,0,1,1,0,0,0,0,-1,2016,4,29,2016,4,29


In [13]:
df.head()

,Gender,Age,Neighborhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSSent,NoShow,WaitingDays,ScheduledDay_Y,ScheduledDay_M,ScheduledDay_D,AppointmentDay_Y,AppointmentDay_M,AppointmentDay_D
0,F,62,JARDIM DA PENHA,0,1,0,0,0,0,No,-1,2016,4,29,2016,4,29
1,M,56,JARDIM DA PENHA,0,0,0,0,0,0,No,-1,2016,4,29,2016,4,29
2,F,62,MATA DA PRAIA,0,0,0,0,0,0,No,-1,2016,4,29,2016,4,29
3,F,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No,-1,2016,4,29,2016,4,29
4,F,56,JARDIM DA PENHA,0,1,1,0,0,0,No,-1,2016,4,29,2016,4,29


## Pandas Get_Dummies Approach

In [13]:
features = ['Gender','Neighborhood','NoShow']

df_gd = pd.concat([df,
                   pd.get_dummies(df[features], drop_first=True,
                   prefix=['G','N','Show'])], axis=1).drop(features, axis=1)

print(df_gd.shape)
df_gd.head()

(110517, 95)


,Age,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSSent,WaitingDays,ScheduledDay_Y,ScheduledDay_M,...,N_SEGURANÇA DO LAR,N_SOLON BORGES,N_SÃO BENEDITO,N_SÃO CRISTÓVÃO,N_SÃO JOSÉ,N_SÃO PEDRO,N_TABUAZEIRO,N_UNIVERSITÁRIO,N_VILA RUBIM,Show_Yes
0,62,0,1,0,0,0,0,0,2016,4,...,0,0,0,0,0,0,0,0,0,0
1,56,0,0,0,0,0,0,0,2016,4,...,0,0,0,0,0,0,0,0,0,0
2,62,0,0,0,0,0,0,0,2016,4,...,0,0,0,0,0,0,0,0,0,0
3,8,0,0,0,0,0,0,0,2016,4,...,0,0,0,0,0,0,0,0,0,0
4,56,0,1,1,0,0,0,0,2016,4,...,0,0,0,0,0,0,0,0,0,0


# Export Data

In [17]:
#LabelEncoder 1 
df_le1.to_csv('data/df_le1.csv',index=False)

In [18]:
#LabelEncoder 2
df_le2.to_csv('data/df_le2.csv',index=False)

In [14]:
#Get_dummies data
df_gd.to_csv('data/df_gd.csv',index=False)

In [52]:
#OneHotEncoder data
df_ohe.to_csv('data/df_ohe.csv',index=False)